In [45]:
!pip3 install sqlalchemy

In [56]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [66]:
import pandas as pd
import os

In [58]:
database_file = os.path.join(os.path.abspath(os.getcwd()), 'data_db')

In [77]:
# engine = create_engine('sqlite:///'+ database_file, convert_unicode=True, echo=True)
engine = create_engine('sqlite:///'+ database_file, echo=True)

In [78]:
db_session = scoped_session(
    sessionmaker(
        autocommit=False,
        autoflush=False,
        bind = engine
    )
)

In [79]:
Base = declarative_base()
Base.query = db_session.query_property()

In [80]:
from sqlalchemy import Column, Integer, Float, text, DateTime
from sqlalchemy.sql.functions import current_timestamp
import time

In [81]:
class Estate(Base):
    __tablename__ = 'estates'
    id = Column(Text, primary_key=True)
    note = Column(Text, unique=False)
    price= Column(Integer, unique=False)
    shop= Column(Text, unique=False)
    place= Column(Text, unique=False)
    prefecture= Column(Text, unique=False)
    city= Column(Text, unique=False)
    station= Column(Text, unique=False)
    route= Column(Text, unique=False)
    work= Column(Text, unique=False)
    area= Column(Float, unique=False)
    buildingarea= Column(Float, unique=False)
    ldk= Column(Text, unique=False)
    buildingyear= Column(Text, unique=False)
    url= Column(Text, unique=False)
    # created= Column(DateTime, nullable=False, server_default=current_timestamp)
    created= Column(DateTime)

    def __init__(self, id=None, note=None, price=None, shop=None, place=None, prefecture=None, city=None, station=None,route=None, work=None, area=None, buildingarea=None, ldk=None, buildingyear=None, url=None):
        self.id = id
        self.note = note
        self.price = price
        self.shop = shop 
        self.place = place 
        self.prefecture = prefecture
        self.city = city 
        self.station = station 
        self.route = route 
        self.work = work
        self.area = area 
        self.buildingarea = buildingarea
        self.ldk = ldk 
        self.buildingyear = buildingyear
        self.url = url



In [82]:
Base.metadata.create_all(bind=engine)

2021-06-12 23:05:09,340 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-12 23:05:09,341 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("estates")
2021-06-12 23:05:09,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-12 23:05:09,357 INFO sqlalchemy.engine.Engine COMMIT


# データの型
マニュアル
Column and Data Types — SQLAlchemy 1.4 Documentation : https://docs.sqlalchemy.org/en/14/core/type_basics.html


In [105]:
def read_data():
    estate_df = pd.read_csv("estate.csv")

    for index, _df in estate_df.iterrows():
        rec = len(db_session.query(Estate).filter(Estate.id==_df['id']).all())
        if rec == 0:
            row = Estate(
                id=_df['id'],
                note=_df['note'],
                price=_df['price'],
                shop=_df['shop'],
                place=_df['place'],
                prefecture=_df['prefecture'],
                city=_df['city'],
                station=_df['station'],
                route=_df['route'],
                work=_df['work'],
                area=_df['area'],
                buildingarea=_df['buildingarea'],
                ldk=_df['ldk'],
                buildingyear=_df['buildingyear'],
                url=_df['url']
                )
            db_session.add(row)
    db_session.commit()

In [106]:
read_data()

estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.id = ?
2021-06-12 23:25:13,517 INFO sqlalchemy.engine.Engine [cached since 337.8s ago] ('b77e8d4a3ca8efe409b5942ef07e5054f3e9965ff3fa997ca89e8dd179e69abb',)
2021-06-12 23:25:13,522 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.

# CRUD操作

In [108]:
db = db_session.query(Estate).all()

2021-06-12 23:26:12,708 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates
2021-06-12 23:26:12,718 INFO sqlalchemy.engine.Engine [cached since 610.1s ago] ()


In [111]:
for row in db:
    print(row.id)

c1746f5fb40176236d146d8811960b0473cb109c339b4ccfcbfd503484a3e57d
c4135b85973e0197b4e60e8a031952ab9fff51288af57083c2ccc03a23af8812
43050007fb98689cbcd2c35e3c8b5037eb6f2240f59e3385eb41ee4afcde85ca
5ce719e3b18480dc92c0c91a02fab53d0a4737534142fb0ed39db0addf132413
76c62d9b592390086bd9d76d5ac0c6b179fafb560241a7b86b1ddd46c35d3411
9104d1f624c104fd15f64284880803b570a26d5ae7740a8cad15a51a705727b9
2e3f3a34c339fbfa7bdd9c0b6e78d654b49da34f55125b5416e93884ee5eb019
0ae98b1e1170733eb67179916faa17eb3412a1b1e1788e78adcdcbb178ad0b45
f9cf50576ddcc3870dbf619057c90fa8c15001098cfaa3d1c47377f314f614a5
3f3085e4baba3f9b726b37ae833163d12e1cd33aec212900a71968f41d3d827f
8dfcab283ebf4aa3700c2bce8adb725ebc6701133724694fceb612ca77404b93
df48d6fac64d6acf7665367855693409e198022b9b882b02582d8ce88d81db21
d143be9d8f1a1c7768303fc7f55a8ce2e5a949f8778ee3bc6bad78fa124990f9
3f3cf7c752cdef3789b59ba85f587303e9ae5faeef9baa06f99505c6e013798b
f5d715f3513ab83b30b861a44527d5b3e59e5180eb5f53ff003e681d27a4b2d3
10712ae16a43c5952a5f71ad9

In [114]:
db = db_session.query(Estate.price, Estate.note).all()

2021-06-12 23:28:01,236 INFO sqlalchemy.engine.Engine SELECT estates.price AS estates_price, estates.note AS estates_note 
FROM estates
2021-06-12 23:28:01,243 INFO sqlalchemy.engine.Engine [generated in 0.00717s] ()


In [116]:
for row in db:
    print(row.note)

西習志野３（北習志野駅） 3480万円
資料請求のみ大歓迎！ご案内も即日ご対応可能です！新規内装リフォーム済み中古…
●２０２１年４月内・外装リフォーム済！●小・中学校安心の徒歩５分圏内！●閑…
習志野台４（習志野駅） 580万円
高野台３ 850万円
松が丘３（高根木戸駅） 980万円
丸山４（馬込沢駅） 990万円
☆船橋市芝山６丁目中古戸建☆
習志野台８（船橋日大前駅） 1980万円
日の出２（船橋駅） 2180万円
夏見台５ 2190万円
前貝塚町（塚田駅） 2880万円
前原東６（津田沼駅） 3199万円
見学できます！一度に周辺の物件や環境も含めてご案内させていただきます♪ほと…
◆土地広々約１３０坪◆広ーい庭があり、ガーデニング・バーベキュー等楽しめま…
本中山１（下総中山駅） 8800万円
前原東３（前原駅） 9980万円
薬円台２（習志野駅） 3680万円
【売主物件】新規内装リフォームお車を2台駐車できるワイドなカーポート
芝山１（飯山満駅） 3980万円
湊町２（船橋駅） 9600万円
☆2路線利用可能！交通便利！☆ファミリー層の多い、閑静な住宅街♪☆スーパー…
☆土地面積広々49.91坪☆駐車2台可能（車種規制有）☆全居室に収納がつい…
◇船橋市栄町１丁目！◇生活利便性の良いエリアに佇む3階建て！◇JR船橋駅・…
資料請求のみ大歓迎！ご案内も即日ご対応可能です！新規内装リフォーム済み中古…
駐車２台可、浴室乾燥機、全居室収納、閑静な住宅地、ＬＤＫ１５畳以上、和室、…
◆築9年！JR船橋駅徒歩圏内の希少物件です！◆南側が抜けているので陽当り◎…
■北総線「小室」駅　徒歩約10分■南東側（幅員約9ｍ）公道（歩道あり）に面…
小屋裏収納ほかたくさんの収納があり、スッキリと暮らせそうです。室内も明るく…
2路線4駅利用可能！カースペース2台分有り☆駅徒歩1分の好立地☆事務所利用…
資料請求のみ大歓迎！ご案内も即日ご対応可能です！平成築の大型戸建！東武アー…
【築10年！オール電化の築浅戸建！】■4ＬＤＫ＋ＷＩＣ＋ウッドデッキ■JR…
対面式キッチン採用のLDKは広々21畳。JR総武線「船橋」駅・「西船橋」駅…
総武線「東船橋」駅徒歩8分土地面積50坪超の広いお庭でガーデニングが出来ま…
資料請求のみ大歓迎！ご案内も即日ご対応可能です！ファミリータイプ

In [127]:
db = db_session.query(Estate).filter(Estate.price>2000).all()
for row in db:
    print(row.price, row.note)

2021-06-12 23:32:01,701 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.price > ?
2021-06-12 23:32:01,703 INFO sqlalchemy.engine.Engine [cached since 160.1s ago] (2000,)
3480 西習志野３（北習志野駅） 3480万円
2180 日の出２（船橋駅） 2180万円
2190 夏見台５ 2190万円
2880 前貝塚町（塚田駅） 2880万円
3199 前原東６（津田沼駅） 3199万円
3980 見学できます！一度に周辺の物件や環境も含めてご案内させていただきます♪ほと…
5880 ◆土地広々約１３０坪◆広ーい庭があり、ガーデニング・バーベキュー等楽しめま…
8800 本中山１（下総中山駅） 8800万円
9980 前原東３（前原駅） 9980万円
3680 薬円台２（習志野駅） 3680万円
3780 【売主物件】新規

In [126]:
db = db_session.query(Estate).filter(Estate.price>2000).limit(5).all()
for row in db:
    print(row.price, row.note)

2021-06-12 23:31:35,180 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.price > ?
 LIMIT ? OFFSET ?
2021-06-12 23:31:35,182 INFO sqlalchemy.engine.Engine [generated in 0.00184s] (2000, 5, 0)
3480 西習志野３（北習志野駅） 3480万円
2180 日の出２（船橋駅） 2180万円
2190 夏見台５ 2190万円
2880 前貝塚町（塚田駅） 2880万円
3199 前原東６（津田沼駅） 3199万円


In [128]:
from sqlalchemy import desc
db = db_session.query(Estate).filter(Estate.price>2000).order_by(desc(Estate.price)).all()
for row in db:
    print(row.price, row.note)

2021-06-12 23:33:25,756 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.price > ? ORDER BY estates.price DESC
2021-06-12 23:33:25,759 INFO sqlalchemy.engine.Engine [generated in 0.00244s] (2000,)
9980 前原東３（前原駅） 9980万円
9600 湊町２（船橋駅） 9600万円
9000 初めてのご来場＆アンケート回答で「ギフトカード2000円分＆選べる千疋屋ス…
8800 本中山１（下総中山駅） 8800万円
7480 ●２世帯住宅　利用可能●キッチンにはIHヒーター搭載（1階はガスコンロにも…
5880 ◆土地広々約１３０坪◆広ーい庭があり、ガーデニング・バーベキュー等楽しめま…
4880 ２沿線以上利用可、ＬＤＫ２０畳以上、土地50坪以上、システムキッチン、対面…
4850 ◆ヘーベル

In [131]:
estate = Estate(id='test', note='test', price=9999, prefecture='', city='')

In [132]:
db_session.add(estate)
db_session.commit()

2021-06-12 23:35:40,088 INFO sqlalchemy.engine.Engine INSERT INTO estates (id, note, price, shop, place, prefecture, city, station, route, work, area, buildingarea, ldk, buildingyear, url, created) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-06-12 23:35:40,090 INFO sqlalchemy.engine.Engine [cached since 860.8s ago] ('test', 'test', 9999, None, None, '', '', None, None, None, None, None, None, None, None, None)
2021-06-12 23:35:40,111 INFO sqlalchemy.engine.Engine COMMIT


In [136]:
db = db_session.query(Estate).filter(Estate.id=='test').all()
for row in db:
    print(row.__dict__)

2021-06-12 23:38:52,742 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.id = ?
2021-06-12 23:38:52,744 INFO sqlalchemy.engine.Engine [cached since 1157s ago] ('test',)
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f06b256a3d0>, 'note': 'test', 'shop': None, 'prefecture': '', 'city': '', 'route': None, 'area': None, 'ldk': None, 'url': None, 'price': 9999, 'id': 'test', 'place': None, 'station': None, 'work': None, 'buil

In [142]:
db = db_session.query(Estate).filter(Estate.id=='test').first()

2021-06-13 00:18:08,339 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-13 00:18:08,340 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.id = ?
 LIMIT ? OFFSET ?
2021-06-13 00:18:08,341 INFO sqlalchemy.engine.Engine [cached since 154.2s ago] ('test', 1, 0)


In [138]:
db.note='実験です！'
db_session.commit()

2021-06-13 00:16:47,282 INFO sqlalchemy.engine.Engine UPDATE estates SET note=? WHERE estates.id = ?
2021-06-13 00:16:47,284 INFO sqlalchemy.engine.Engine [generated in 0.00168s] ('実験です！', 'test')
2021-06-13 00:16:47,297 INFO sqlalchemy.engine.Engine COMMIT


In [146]:
db_session.query(Estate).filter(Estate.id=='test').delete()

2021-06-13 00:20:47,706 INFO sqlalchemy.engine.Engine DELETE FROM estates WHERE estates.id = ?
2021-06-13 00:20:47,708 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ('test',)


1

In [147]:
db = db_session.query(Estate).filter(Estate.id=='test').all()
len(db)


2021-06-13 00:21:55,578 INFO sqlalchemy.engine.Engine SELECT estates.id AS estates_id, estates.note AS estates_note, estates.price AS estates_price, estates.shop AS estates_shop, estates.place AS estates_place, estates.prefecture AS estates_prefecture, estates.city AS estates_city, estates.station AS estates_station, estates.route AS estates_route, estates.work AS estates_work, estates.area AS estates_area, estates.buildingarea AS estates_buildingarea, estates.ldk AS estates_ldk, estates.buildingyear AS estates_buildingyear, estates.url AS estates_url, estates.created AS estates_created 
FROM estates 
WHERE estates.id = ?
2021-06-13 00:21:55,580 INFO sqlalchemy.engine.Engine [cached since 3740s ago] ('test',)


0